### Notebook to determine which point of data_traite from dataAnalysis presents the biggest permeability value

In [1]:
#Necessary imports

from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import chardet
import csv
import os

In [2]:
path = 'C:/Users/alexa/2A/MOLONARI1D/dataAnalysis/data_traite'

print("Les points de mesures, en température et en pression, sont les suivants :")

os.listdir(path)[1:]

Les points de mesures, en température et en pression, sont les suivants :


['point13_pression_traité.csv',
 'point13_temperature_traité.csv',
 'point14_pression_traité.csv',
 'point14_temperature_traité.csv',
 'point34_pression_traité.csv',
 'point34_temperature_traité.csv',
 'point35_pression_traité.csv',
 'point35_temperature_traité.csv',
 'point36_pression_traité.csv',
 'point36_temperature_traité.csv',
 'point46_pression_traité.csv',
 'point46_temperature_traité.csv',
 'point47_pression_traité.csv',
 'point47_temperature_traité.csv',
 'point48_pression_traité.csv',
 'point48_temperature_traité.csv',
 'point52_pression_traité.csv',
 'point52_temperature_traité.csv',
 'point53_pression_traité.csv',
 'point53_temperature_traité.csv']

In [5]:
#Point considéré

point_pression = 'point13_pression_traité.csv'
point_temperature = 'point13_temperature_traité.csv'

#Définition d'un dictionnaire pour les données 

dico = {}
dico['chemin pression'] = os.path.join(path, point_pression)
dico['chemin temperature'] = os.path.join(path, point_temperature)

#Lecture des données

def read_csv (chemin_fichier):
    #Detecter separateur
    with open(chemin_fichier, 'r') as file:
        sniffer = csv.Sniffer()
        sample_data = file.read(1024)
        detecter_separateur = (sniffer.sniff(sample_data).delimiter)

    if "Titre" in open(chemin_fichier).readline():
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur, skiprows=1)
    else:
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur)

    return data_frame 

pression = read_csv(dico['chemin pression'])
temperature = read_csv(dico['chemin temperature'])

dico['pression'] = pression
dico['temperature'] = temperature

print(dico['pression'].head())
print(dico['temperature'].head())

dico['temperature'].columns = ['dates', 'T_10', 'T_20', 'T_30', 'T_40']

              dates  temperature_stream        dH
0  23-07-2015 14:00              18.224  0.085864
1  23-07-2015 14:15              18.081  0.086153
2  23-07-2015 14:30              18.057  0.086189
3  23-07-2015 14:45              18.057  0.086257
4  23-07-2015 15:00              18.176  0.086072
              dates  Temp_profondeur_10  Temp_profondeur_20  \
0  23-07-2015 14:00              18.319              17.938   
1  23-07-2015 14:15              18.224              17.653   
2  23-07-2015 14:30              18.105              17.558   
3  23-07-2015 14:45              18.081              17.534   
4  23-07-2015 15:00              18.057              17.510   

   Temp_profondeur_30  Temp_profondeur_40  
0              18.247              18.390  
1              17.915              17.986  
2              17.796              17.796  
3              17.748              17.748  
4              17.724              17.724  


In [11]:
#On va maintenant compute la MCMC pour les données de pression et de température, pour trouver une valeur de 
#perméabilité pour chaque point de mesure

#On commence par créer des listes de tuple pour les données de pression et de température

data_pression = list(zip(dico['pression']['dates'], list(zip( dico['pression']['dH'], dico['pression']['temperature_stream']))))
data_temperature = list(zip(dico['temperature']['dates'], zip(dico['temperature']['T_10'], dico['temperature']['T_20'], dico['temperature']['T_30'], dico['temperature']['T_40'])))

print (data_pression[:5])
print (data_temperature[:5])

[('23-07-2015 14:00', (0.0858643254074198, 18.224)), ('23-07-2015 14:15', (0.0861528697161693, 18.081)), ('23-07-2015 14:30', (0.0861893984470073, 18.057)), ('23-07-2015 14:45', (0.0862567747756304, 18.057)), ('23-07-2015 15:00', (0.0860715126342083, 18.176))]
[('23-07-2015 14:00', (18.319, 17.938, 18.247, 18.39)), ('23-07-2015 14:15', (18.224, 17.653, 17.915, 17.986)), ('23-07-2015 14:30', (18.105, 17.558, 17.796, 17.796)), ('23-07-2015 14:45', (18.081, 17.534, 17.748, 17.748)), ('23-07-2015 15:00', (18.057, 17.51, 17.724, 17.724))]


In [12]:
#On initialise les paramètres de la MCMC et de l'objet column à partir d'un dico

Z_bottom = .4
depth_sensors = [0.1, 0.2, 0.3, Z_bottom]

col_dict = {
	"river_bed": .4, 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": data_pression,
    "T_measures": data_temperature,
    "sigma_meas_P": 0.01, #float
    "sigma_meas_T": 0.1, #float
}

col = Column.from_dict(col_dict)

priors = {
    "moinslog10IntrinK": ((10, 16), .01), # (intervalle, sigma)
    "n": ((.01, .25), .01),
    "lambda_s": ((1, 10), .1),
    "rhos_cs": ((1e6,1e7), 1e5),
}

all_priors = [
    ['Couche 1', Z_bottom, priors],
]

#On va maintenant lancer la MCMC

